In [200]:
import os
import requests
import glob
import zipfile
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime


#Prepare:
unzip_local = os.path.join(os.getcwd(),''.join(['Data ', datetime.now().strftime('%d-%m-%Y')]))
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
with zipfile.ZipFile("datasource.zip", 'r') as zip_ref:
    zip_ref.extractall(unzip_local)

open ("dealership_logfile.txt","a")
open ("dealership_transformed_data.csv","a")
logfile    = "dealership_logfile.txt"            
targetfile = "dealership_transformed_data.csv" 


#Extract:
def extract_from_csv(file):
    df = pd.read_csv(file)
    return df

def extract_from_json(file):
    df = pd.read_json(file,lines=True)
    return df

def extract_from_xml(file):
    df = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])
    tree = ET.parse(file)
    root = tree.getroot()
    for car in root:
        car_model = car.find('car_model').text
        year_of_manufacture = int(car.find('year_of_manufacture').text)
        price = float(car.find('price').text)
        fuel = car.find('fuel').text
        df = pd.concat([df, pd.DataFrame.from_dict({'car_model':[car_model], 'year_of_manufacture':[year_of_manufacture], 'price':[price], 'fuel':[fuel]})], ignore_index=True)
    return df

def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    for csvfile in glob.glob(os.path.join(unzip_local,'*[!dealership_transformed_data].csv')):
        extracted_data = pd.concat([extracted_data,extract_from_csv(csvfile)], ignore_index=True)
    for jsonfile in glob.glob(os.path.join(unzip_local,'*.json')):
        extracted_data = pd.concat([extracted_data,extract_from_json(jsonfile)], ignore_index=True)    
    for xmlfile in glob.glob(os.path.join(unzip_local,'*.xml')):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True)
    return extracted_data


#Transform:
def transform(data):
    data = data.sort_values(by=['price'])
    data['price'] = round(data.price, 2)
    r = requests.get('https://api.exchangerate-api.com/v4/latest/USD').text
    exrate = eval(r)['rates']['VND']
    data['VND Price (million)'] = data.get('VND Price (million)',round(data.price*exrate/1000000,2))
    data['price'] = data['price'].map('${:,.2f}'.format)
    data['VND Price (million)'] = data['VND Price (million)'].map('VND {:,.2f}'.format)
    data['car_model'] = data['car_model'].str.capitalize()
    data.rename(columns={'car_model':'Model','year_of_manufacture':'Year of Manufacture','price':'USD Price','fuel':'Fuel type'},inplace=True)
    data = data[['Model','Year of Manufacture','Fuel type','USD Price','VND Price (million)']]
    data = data.reset_index(drop=True)
    return data


#Load:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

    
#Logging Entries:
def log(mes):
    now = datetime.now()
    timestamp = now.strftime('%d-%h-%Y %H:%M:%S')
    with open (logfile, 'a') as f:
        f.write(timestamp + ': ' + mes + '\n')


#Running ETL Process:
log("ETL Process Started")

log("Extract Phase Started")
extracted_data = extract()
log("Extract Phase Ended")

log("Transform Phase Started")
transformed_data = transform(extracted_data)
log("Transform Phase Ended")

log("Load Phase Started")
load(targetfile, transformed_data)
log("Load Phase Ended")

log("ETL Process Ended")

--2023-02-02 18:03:09--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: 'datasource.zip.2'

     0K ....                                                  100% 1.24G=0s

2023-02-02 18:03:10 (1.24 GB/s) - 'datasource.zip.2' saved [4249/4249]

